In [ ]:
# 0
# if you haven't installed wget, uncomment the below command and run this cell
#!pip install wget

In [65]:
# 1
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import time
import wget
import os

In [113]:
urls

['https://guides.library.fresnostate.edu/agbs120',
 'https://guides.library.fresnostate.edu/agbs150',
 'https://guides.library.fresnostate.edu/agri100h',
 'https://guides.library.fresnostate.edu/agmarketing',
 'https://guides.library.fresnostate.edu/agriculture',
 'https://guides.library.fresnostate.edu/ama',
 'https://guides.library.fresnostate.edu/animalsciences',
 'https://guides.library.fresnostate.edu/armenianstudies',
 'https://guides.library.fresnostate.edu/artanddesign',
 'https://guides.library.fresnostate.edu/ai',
 'https://guides.library.fresnostate.edu/becoming-a-librarian',
 'https://guides.library.fresnostate.edu/c.php?g=289102',
 'https://guides.library.fresnostate.edu/biol102',
 'https://guides.library.fresnostate.edu/biology1a',
 'https://guides.library.fresnostate.edu/biology',
 'https://guides.library.fresnostate.edu/business',
 'https://guides.library.fresnostate.edu/swot',
 'https://guides.library.fresnostate.edu/califstats',
 'https://guides.library.fresnostate.ed

In [66]:
# 2
####
# where are you scraping?
# urls = "https://guides.library.fresnostate.edu/agbs120" # if using individual website

## if list of URLs
# Open the file in read mode
with open('libguides-url-list.txt', 'r') as file:
    urls = file.read()
file.close()    
urls = urls.split('\n')


####
# where do you want to save locally?

# save_loc is redundant, but a reminder that you should have locally folders that reflect the years, because that's where below is saving to
save_loc = "libguides" 

# what kind of prefix do you want on your files
save_prefix = "libguides_" 

In [115]:
urls_saved = [] 

In [118]:
for url in urls[0:2]:
    if url in urls_saved:
        print("in saved, do nothing1")
    else:
        webpage = requests.get(url)
        soup = BeautifulSoup(webpage.content, 'html.parser')
        for link in soup.find_all('a')[0:16]:
            data_target = link.get('href')
    #        print(data_target)
            time.sleep(.25) # be kind, don't look like a DDOS attack
            if ("guides.library.fresnostate.edu") in data_target:
        #        print("is libguide", data_target)
                if data_target[-1]=="/":
        #            print("strip char")
                    data_target = data_target[:-1]
                    
                if data_target in urls_saved:
                    print("in saved, do nothing2") 
                else:
                    if ("prf.php" in data_target) or ("sb.php" in data_target) or ("database/" in data_target) or ("srch.php" in data_target):
                        print("profile/subject/database page, do nothing:", data_target)
                    else:
                        urls_saved.append(data_target)
                        wget_url = data_target
                        wget_save = "./libguides/"+save_prefix+data_target.replace("https://guides.library.fresnostate.edu","").replace("/c.php?g=","").replace("/","").replace("&p=","p_")
                        if not((wget_save[-3:]=="htm") or (wget_save[-4:]=="html")):
                            wget_save = wget_save+".html"
                        
                        if os.path.exists(wget_save):
                            print(wget_save+" already exists. Did NOT save.")
                        else:
                            print("SAVING "+ wget_url+ " at local location: "+wget_save)
                            wget.download(wget_url,wget_save)
                            time.sleep(.25) # be kind, don't look like a DDOS attack

print("fin.")

in saved, do nothing1
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing1
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
in saved, do nothing2
fin.


In [88]:
urls_saved = [] 

In [105]:
for link in soup.find_all('a'):
    data_target = link.get('href')
#    print(data_target)
    if ("guides.library.fresnostate.edu") in data_target:
#        print("is libguide", data_target)
        if data_target[-1]=="/":
#            print("strip char")
            data_target = data_target[:-1]
            
        if data_target in urls_saved:
            print("in saved, do nothing") 
        else:
            if ("prf.php" in data_target) or ("sb.php" in data_target) or ("database/" in data_target) or ("srch.php" in data_target):
                print("profile/subject/database page, do nothing:", data_target)
            else:
                wget_url = data_target
                wget_save = "./libguides/"+save_prefix+data_target.replace("https://guides.library.fresnostate.edu","").replace("/c.php?g=","").replace("/","").replace("&p=","p_")
                if not((wget_save[-3:]=="htm") or (wget_save[-4:]=="html")):
                    wget_save = wget_save+".html"
                
                if os.path.exists(wget_save):
                    print(wget_save+" already exists. Did NOT save.")
                else:
                    print("SAVING "+ wget_url+ " at local location: "+wget_save)
                    wget.download(wget_url,wget_save)
                    time.sleep(.25) # be kind, don't look like a DDOS attack
                    urls_saved.append(data_target)

print("fin.")

in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
in saved, do nothing
profile/subject/database page, do nothing: //guides.library.fresnostate.edu/prf.php?id=592a381f-7cdb-11ed-9922-0ad758b798c3
profile/subject/database page, do nothing: https://guides.library.fresnostate.edu/sb.php?subject_id=58523
profile/subject/database page, do nothing: https://guides.library.fresnostate.edu/sb.php?subject_id=58539
in saved, do nothing
profile/subject/database page, do nothing: https://guides.library.fresnostate.edu/database/data-axle-reference-solutions
profile/subject/database page, do nothing: https://guides.library.fresnostate.edu/database/checkpoint-edge
profile/subject/database page, do nothing: https://guides.library.fresnostate.edu/database/privco
profile/subject/database page, do nothing: h

In [97]:
urls_saved

['https://guides.library.fresnostate.edu',
 'https://guides.library.fresnostate.edu/agbs120',
 'https://guides.library.fresnostate.edu/c.php?g=289058&p=1928695']

In [101]:
data_target

'//guides.library.fresnostate.edu/prf.php?id=592a381f-7cdb-11ed-9922-0ad758b798c3'

In [43]:
with open('page_content.html', 'w') as fid:
    fid.write(webpage.text)
fid.close()

In [73]:
wget_url = data_target
wget_save = "./libguides/"+save_prefix+data_target.replace("https://guides.library.fresnostate.edu/c.php?g=","").replace("&p=","p_")
if not((wget_save[-3:]=="htm") or (wget_save[-4:]=="html")):
    wget_save = wget_save+".html"

In [74]:
wget_save

'./libguides/libguides_289058p_1928695.html'

In [75]:
wget_url

'https://guides.library.fresnostate.edu/c.php?g=289058&p=1928695'

In [77]:
wget.download(wget_url,wget_save)

'./libguides/libguides_289058p_1928695.html'

In [13]:
# 3
# second for loop has [0:16] at the end of range for testing.  delete range to save full list
for year in years:
    url = base_url+data_url+year_base+year+session_id 
    webpage = requests.get(url)
    soup = BeautifulSoup(webpage.content, 'html.parser')


    if os.path.isdir("./"+year):
        # notice output path has "year" as a variable indicating folders
        output_path = "./"+year+"/"+save_prefix+year+"_"+".html"
    
        if os.path.exists(output_path):
            print(output_path+" already exists. Did NOT save.")
        else:
            # saving the html file
            print("saving "+output_path)
    
            with open(output_path, 'wb') as file:
                file.write(webpage.content) 
            
        # saving files linked on original html page which meet file type requirement
        for link in soup.find_all('a')[0:16]:  
            data_target = link.get('href')
            # data_target[-3:]
            if any(extension == data_target[-3:] for extension in file_type):
                wget_url = base_url+data_target
                wget_save = "./"+year+"/"+save_prefix+data_target.replace("/","-")
    
                if os.path.exists(wget_save):
                    print(wget_save+" already exists. Did NOT save.")
                else:
                    print("SAVING "+ wget_url+ " at local location: "+wget_save)
                    wget.download(wget_url,wget_save)
                    time.sleep(.25) # be kind, don't look like a DDOS attack
    else:
        print("\n!!! save directory "+year+" does NOT exist. please create\n")
    
    
print("fin.")

    

./1995/ipeds_1995_.html already exists. Did NOT save.
./1995/ipeds_data-ic9596_A.zip already exists. Did NOT save.
./1995/ipeds_data-ic9596_A_Data_Stata.zip already exists. Did NOT save.
./1995/ipeds_data-ic9596_A_SPS.zip already exists. Did NOT save.
./1996/ipeds_1996_.html already exists. Did NOT save.
./1996/ipeds_data-ic9697_A.zip already exists. Did NOT save.
./1996/ipeds_data-ic9697_A_Data_Stata.zip already exists. Did NOT save.
./1996/ipeds_data-ic9697_A_SPS.zip already exists. Did NOT save.

!!! save directory 1997 does NOT exist. please create

./1998/ipeds_1998_.html already exists. Did NOT save.
SAVING https://nces.ed.gov/ipeds/datacenter/data/IC98hdac.zip at local location: ./1998/ipeds_data-IC98hdac.zip
SAVING https://nces.ed.gov/ipeds/datacenter/data/IC98hdac_Data_Stata.zip at local location: ./1998/ipeds_data-IC98hdac_Data_Stata.zip
SAVING https://nces.ed.gov/ipeds/datacenter/data/IC98hdac_SPS.zip at local location: ./1998/ipeds_data-IC98hdac_SPS.zip
fin.
